In [1]:
import scrapy

In [2]:
class Spider12(scrapy.Spider):
    name = "Spider12"
    allowed_domains = ['pagina12.com.ar']
    custom_settings = {'FEED_FORMAT': 'json',
                       'FEED_EXPORT_ENCODING': 'utf-8',
                       'FEED_URI': 'resultados.json',
                       'DEPTH_LIMIT': 2}
    
    start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/ciencia',
                  'https://www.pagina12.com.ar/secciones/el-mundo']
    
    def parse(self, response):
        #Listado de notas
        notas = response.xpath('//div[@class="article-item__content"]//a/@href').getall()
        for nota in notas:
            yield response.follow(nota, callback=self.parse_nota)
            
        #Link a la siguiente pagina
        next_page = response.xpath('//a[@class="next"]/@href').get()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
                                   
        
            
    def parse_nota(self, response):
    #Parsear la nota
        
        #Titulo
        titulo = response.xpath('//h1/text()').get()
        #Date
        date = response.xpath('//div[@class="date modification-date"]/span/time/@datetime').get()
        #Volanta
        volanta = response.xpath('//h3/text()').get()
        #Resumen
        try:
            resume = response.xpath('//h2/text()').get()
        except Exception as e:
            resume = None
            print('No tiene resumen')
        #Cuerpo
        cuerpo = response.xpath('//div[@class="article-main-content article-text "]/p//text()').get()
        
        yield {'url': response.url,
               'titulo': titulo,
               'date': date,
               'volanta': volanta,
               'resume': resume,
               'cuerpo': cuerpo}

In [3]:
from scrapy.crawler import CrawlerProcess

In [4]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

2022-12-15 18:02:43 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2022-12-15 18:02:43 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Windows-10-10.0.22621-SP0
2022-12-15 18:02:43 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 2, 'FEED_EXPORT_ENCODING': 'utf-8'}
2022-12-15 18:02:43 [py.warnings] WARNING: C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_I